In [1]:
!where python

c:\Users\xingy\OneDrive\Documents\Personal\Projects\AI Bootcamp\hr_bot\venv\Scripts\python.exe
C:\Users\xingy\AppData\Local\Programs\Python\Python39\python.exe


In [8]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import tiktoken

# Load .env into Environment Varaibles
load_dotenv('.env')

# Define default model
model_default = "gpt-4o"

# Instantiate the OpenAI client
client = OpenAI()

In [9]:
def get_completion_by_messages(messages, model=model_default, temperature=0, top_p=1.0, max_tokens=1024, n=1):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        n=1
    )
    return response.choices[0].message.content

messages=[
{"role": "system", "content": "You are a helpful assistant."},
{"role": "user", "content": "List some Fun Activities"},
{"role": "assistant", "content": "Spa, Hiking, Surfing, and Gaming"},
{"role": "user", "content": "Which are healthy?"}
]

get_completion_by_messages(messages)

'Several fun activities are also great for your health. Here are a few:\n\n1. **Hiking**: Excellent for cardiovascular health, muscle strength, and mental well-being.\n2. **Swimming**: Great for a full-body workout and low-impact exercise.\n3. **Cycling**: Good for cardiovascular fitness, muscle strength, and joint mobility.\n4. **Yoga**: Enhances flexibility, strength, and mental clarity.\n5. **Dancing**: Provides a fun way to improve cardiovascular health, coordination, and muscle tone.\n6. **Running or Jogging**: Boosts cardiovascular health, endurance, and mental health.\n7. **Tennis or Badminton**: Improves hand-eye coordination, cardiovascular health, and agility.\n8. **Rock Climbing**: Builds strength, endurance, and problem-solving skills.\n9. **Kayaking or Canoeing**: Strengthens upper body muscles and improves cardiovascular health.\n10. **Gardening**: Offers moderate physical activity and can reduce stress.\n\nThese activities not only provide physical benefits but also cont

In [10]:
# This function is for calculating the tokens given the "message"
# ⚠️ This is simplified implementation that is good enough for a rough estimation

import tiktoken

def count_tokens(text):
    encoding = tiktoken.encoding_for_model(model_default)
    return len(encoding.encode(text))

def count_tokens_from_message_rough(messages):
    encoding = tiktoken.encoding_for_model(model_default)
    value = ' '.join([x.get('content') for x in messages])
    return len(encoding.encode(value))

In [12]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./data/DRAFT 1_Maternity Leave Information.docx")
docs = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=500,
    chunk_overlap=50,
    length_function=count_tokens
)

splitted_documents = text_splitter.split_documents(docs)
len(splitted_documents)

5

In [20]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')

db = Chroma.from_documents(splitted_documents, embeddings_model, persist_directory="./chroma_db")

In [21]:
print(db._collection.count())

5


In [24]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

qa_chain = RetrievalQA.from_chain_type(
    ChatOpenAI(model='gpt-3.5-turbo'),
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True
)

response = qa_chain.invoke("Am I eligible for maternity leave if I am not a citizen?")
response

{'query': 'Am I eligible for maternity leave if I am not a citizen?',
 'result': 'Yes, you are eligible for maternity leave even if you are not a citizen, as long as you meet the other eligibility criteria specified for maternity leave as an officer.',
 'source_documents': [Document(page_content='Officers whose NPL end after the first 8 weeks from the date of the birth of the child would be granted the remainder of the 4 or 8 weeks of EML, to be taken within 12 months from the date of the birth of the child. Any remaining EML not taken within this period will be forfeited.\n\n\n\n\n\nProcedure\n\nOfficers are to inform their HR Business Partner of their EDD and the planned start date of your ML (can be up to 4 weeks before EDD or upon delivery, please see diagram below).\n\nSubmit to HR Business Partner their Doctor\'s Certification indicating the officer\'s EDD\n\nInform your HR Business Partner of your ML plan (length of period to be taken consecutively after confinement)\n\nThe firs

In [25]:
len(response['source_documents'])

2

In [26]:
response['source_documents']

[Document(page_content='Officers whose NPL end after the first 8 weeks from the date of the birth of the child would be granted the remainder of the 4 or 8 weeks of EML, to be taken within 12 months from the date of the birth of the child. Any remaining EML not taken within this period will be forfeited.\n\n\n\n\n\nProcedure\n\nOfficers are to inform their HR Business Partner of their EDD and the planned start date of your ML (can be up to 4 weeks before EDD or upon delivery, please see diagram below).\n\nSubmit to HR Business Partner their Doctor\'s Certification indicating the officer\'s EDD\n\nInform your HR Business Partner of your ML plan (length of period to be taken consecutively after confinement)\n\nThe first 8 weeks of the ML (known in ESS as Maternity Leave) has to be taken consecutively.\n\n\n\nThe remaining 8 weeks of the ML (known in ESS as Extended Maternity Leave), or 4 weeks of the no pay ML (known in ESS as No Pay Maternity Leave) can be taken flexibly over a \u200b12